In [1]:
import findspark
findspark.init('/Users/lmh/Programs/spark-2.2.0-bin-hadoop2.7')
from pyspark.sql import SparkSession 
from pyspark.sql import Row
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from matplotlib import pyplot as plt
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import mean_squared_error
from math import sqrt

In [2]:
rdd = sc.textFile('./csv/ratings_Sports_withID.csv')
rdd = rdd.map(lambda x: x.split(','))
rdd = rdd.filter(lambda x: int(x[3]) < 40000)
rdd = rdd.filter(lambda x: int(x[0]) < 8000)
rdd.take(10)

[[u'1412', u'B00004YVAJ', u'5.0', u'15010', u'A35UHVTCH8150C'],
 [u'1412', u'B00004YVAJ', u'5.0', u'20982', u'A2Q7C95JCR0HV7'],
 [u'1412', u'B00004YVAJ', u'3.0', u'15465', u'A1ET2OJ15PESUA'],
 [u'1412', u'B00004YVAJ', u'5.0', u'21070', u'A2PVOGZSGBKCCN'],
 [u'1412', u'B00004YVAJ', u'5.0', u'23138', u'APMAMP7QL4JJ7'],
 [u'1412', u'B00004YVAJ', u'5.0', u'25468', u'A3CZ2AEU7ONYC8'],
 [u'1412', u'B00004YVAJ', u'4.0', u'3096', u'A1MDHR1E6FH490'],
 [u'1412', u'B00004YVAJ', u'5.0', u'33574', u'AR19W42BFDGJX'],
 [u'1412', u'B00004YVAJ', u'5.0', u'5476', u'AAA4HSDSUM369'],
 [u'1412', u'B00004YVAJ', u'5.0', u'16778', u'A2UHQPRBNDR87J']]

In [3]:
user = rdd.map(lambda x: x[3])
item = rdd.map(lambda x: x[0])

In [4]:
def mergedic(x, y):
    x.update(y)
    return x

In [5]:
userd = user.distinct().zipWithIndex()
userd = userd.map(lambda x: {x[0]: x[1] + 1})
user_dic = userd.reduce(mergedic)

In [6]:
itemd = item.distinct().zipWithIndex()
itemd = itemd.map(lambda x: {x[0]: x[1] + 1})
item_dic = itemd.reduce(mergedic)

In [7]:
cb = rdd.map(lambda (item, itemname, rating, user, username): \
                    [username, itemname, rating, user_dic[user], item_dic[item]])

In [8]:
cbrdd = cb.map(lambda x: Row(username = x[0],
                             itemname = x[1],
                             userid = int(x[3]),
                             rating = float(x[2]),                            
                             itemid = int(x[4])))

In [9]:
df = spark.createDataFrame(cbrdd)

In [10]:
tdf = df.groupby('userid').agg({'rating': 'mean'})
df = tdf.join(df, tdf.userid == df.userid).drop(tdf.userid)
df = df.withColumnRenamed('avg(rating)', 'usermean')

In [11]:
tdf = df.groupby('itemid').agg({'rating': 'mean'})
df = tdf.join(df, tdf.itemid == df.itemid).drop(tdf.itemid)
df = df.withColumnRenamed('avg(rating)', 'itemmean')

In [12]:
df.show()

+-----------------+--------+------+----------+------+------+--------------+
|         itemmean|usermean|itemid|  itemname|rating|userid|      username|
+-----------------+--------+------+----------+------+------+--------------+
|              5.0|     5.0|    26|B00FSVMQ76|   5.0|   584|A3OQX4MD953LTJ|
|              1.0|     1.0|    29|B00079YU46|   1.0|   986|A312JTRQVEA4G7|
|              5.0|     5.0|   474|B0085ADXP6|   5.0|   181|A2ARTZ9VDPROVV|
|4.476190476190476|     4.0|    65|B00004YVAJ|   4.0|  1038|A1OKXOCJV6YXZO|
|4.476190476190476|     5.0|    65|B00004YVAJ|   5.0|   909|A23OYTJOL7S449|
|4.476190476190476|     5.0|    65|B00004YVAJ|   5.0|   271|A2UHQPRBNDR87J|
|4.476190476190476|     3.0|    65|B00004YVAJ|   3.0|   865|A1ET2OJ15PESUA|
|4.476190476190476|     5.0|    65|B00004YVAJ|   5.0|   901|A2SSHC3B4J1YOL|
|4.476190476190476|     5.0|    65|B00004YVAJ|   5.0|   250| AAA4HSDSUM369|
|4.476190476190476|     5.0|    65|B00004YVAJ|   5.0|  1155|A23U9SF6JCZ78C|
|4.476190476

In [13]:
pdf = df.toPandas()

In [14]:
pdf

,itemmean,usermean,itemid,itemname,rating,userid,username
0,5.00000,5.0,26,B00FSVMQ76,5.0,584,A3OQX4MD953LTJ
1,1.00000,1.0,29,B00079YU46,1.0,986,A312JTRQVEA4G7
2,5.00000,5.0,474,B0085ADXP6,5.0,181,A2ARTZ9VDPROVV
3,4.47619,4.0,65,B00004YVAJ,4.0,1038,A1OKXOCJV6YXZO
4,4.47619,5.0,65,B00004YVAJ,5.0,909,A23OYTJOL7S449
5,4.47619,5.0,65,B00004YVAJ,5.0,271,A2UHQPRBNDR87J
6,4.47619,3.0,65,B00004YVAJ,3.0,865,A1ET2OJ15PESUA
7,4.47619,5.0,65,B00004YVAJ,5.0,901,A2SSHC3B4J1YOL
8,4.47619,5.0,65,B00004YVAJ,5.0,250,AAA4HSDSUM369
9,4.47619,5.0,65,B00004YVAJ,5.0,1155,A23U9SF6JCZ78C


In [15]:
n_users = pdf.userid.unique().shape[0]
n_items = pdf.itemid.unique().shape[0]
print 'Number of users = ' + str(n_users) + ' | Number of movies = ' + str(n_items)

Number of users = 1156 | Number of movies = 695


In [16]:
data_matrix = np.zeros((n_users, n_items))
usermean = np.zeros(n_users)
itemmean = np.zeros(n_items)
for line in pdf.itertuples():
    data_matrix[line[6] - 1, line[3] - 1] = line[5]
    usermean[line[6] - 1] = line[2]
    itemmean[line[3] - 1] = line[1]

In [17]:
user_similarity = pairwise_distances(data_matrix, metric='cosine')
item_similarity = pairwise_distances(data_matrix.T, metric='cosine')

In [18]:
for i in range(n_users):
    user_similarity[i][i] = -1
    
for i in range(n_items):
    item_similarity[i][i] = -1

In [19]:
k_user = 20
k_item = 20
alpha = 0.002
beta = 0.002

In [20]:
user_kNN = np.argsort(user_similarity, axis = 1)[:, -k_user:]
item_kNN = np.argsort(item_similarity, axis = 1)[:, -k_item:]

In [21]:
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten()
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

In [22]:
prediction = np.zeros((n_users, n_items))
for i in range(n_users):
    for j in range(n_items):
        prediction[i][j] = (usermean[i] + itemmean[j]) / 2.0
        for k in range(k_user):
            prediction[i][j] = prediction[i][j] + \
            + alpha * (user_similarity[i][k] * (data_matrix[k, j] - itemmean[j])) 
                
        for k in range(k_item):
            prediction[i][j] = prediction[i][j] + \
            + beta * (user_similarity[i][k] * (data_matrix[k, j] - itemmean[j])) 
                    
rmse_record = rmse(prediction, data_matrix)
print 'RMSE for alpha = ' + str(alpha) + ', beta = ' + str(beta) + ' = ' + str(rmse_record)

RMSE for alpha = 0.002, beta = 0.002 = 0.499148112264


In [23]:
def recUser(username, n):
    global prediction, pdf
    for line in pdf.itertuples():
        if line[7] == username:
            userid = line[6]
            break
    
    itemids = np.argsort(prediction[userid])[-n:]
    print 'Recommend User ' + username + ' following items:'
    for num in range(n):
        for line in pdf.itertuples():
            if line[3] == itemids[num] + 1:
                print line[4]
                break

In [24]:
def recItem(itemname, n):
    global prediction, pdf
    for line in pdf.itertuples():
        if line[4] == itemname:
            itemid = line[3]
            break
    
    userids = np.argsort(prediction[itemid])[-n:]
    print 'Recommend Item ' + itemname + ' following userss:'
    for num in range(n):
        for line in pdf.itertuples():
            if line[6] == userids[num] + 1:
                print line[7]
                break

In [25]:
recUser('A35UHVTCH8150C', 5)

Recommend User A35UHVTCH8150C following items:
B00I4WTQZQ
B001TI4XQO
B002N4MLRQ
B00418QFKG
B0000VMYEO


In [26]:
recItem('B00004YVAJ', 5)

Recommend Item B00004YVAJ following userss:
A2Q4K02P0WOK0N
A3LA9EJXWPWOW
A3HKM5NFVR7MX7
A2MIB7Y8X04WG5
AHXQ51RM5LXAV
